In [1]:
import pandas as pd

df = pd.read_csv("C:\\MAIN\\NYCDSA\\Web_Scraping_Project\\Reuters\\reuters_news.csv")

In [7]:
df.head()

,body,classification,timestamp,title
0,DETROIT (Reuters) - A U.S. regulator said it w...,Business News,2020-01-23 16:38:00,U.S. regulator to review recommendation on Fia...
1,"(Reuters) - Procter & Gamble Co (,) on Thursda...",Business News,2020-01-23 12:22:00,P&G sales growth shy of forecasts as slowing b...
2,"(Reuters) - Tesla Inc (,) has overtaken German...",Technology News,2020-01-23 14:23:00,Tesla overtakes Volkswagen as world's second m...
3,BERLIN (Reuters) - Seven U.S. bombs from World...,Technology News,2020-01-23 16:20:00,Tesla's German factory gets off to an explosiv...
4,"(Reuters) - Xerox Holdings Corp (,) said on Th...",Business News,2020-01-23 14:35:00,Xerox nominates new HP board after buyout rebuffs


In [9]:
df.shape

(97, 4)